<a href="https://colab.research.google.com/github/andry300000/image_2025/blob/main/lab_7_8/lab_7_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
from collections import Counter
import os

1. Выполните сохранение монохромного изображения в виде текстового файла.

In [68]:
img_path = 'image.png'
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

np.savetxt('image.txt', img, fmt='%d')
pixel_storage_size = os.path.getsize('image.txt')

2. Реализуйте алгоритм вейвлет-преобразования Хаара для изображения.

In [71]:
def haar(image):
    rows, cols = image.shape
    image_float = image.astype(np.float64)
    row_transform = np.zeros_like(image_float)

    for i in range(rows):
        for j in range(0, cols - 1, 2):
            row_transform[i, j // 2] = (image_float[i, j] + image_float[i, j + 1]) / 2
            row_transform[i, (j // 2) + cols // 2] = (image_float[i, j] - image_float[i, j + 1]) / 2

    result = np.zeros_like(row_transform)
    for j in range(cols):
        for i in range(0, rows - 1, 2):
            result[i // 2, j] = (row_transform[i, j] + row_transform[i + 1, j]) / 2
            result[(i // 2) + rows // 2, j] = (row_transform[i, j] - row_transform[i + 1, j]) / 2

    LL = result[:rows // 2, :cols // 2]
    LH = result[:rows // 2, cols // 2:]
    HL = result[rows // 2:, :cols // 2]
    HH = result[rows // 2:, cols // 2:]
    return LL, LH, HL, HH

LL, LH, HL, HH = haar(img)

3. Выполните квантование высокочастотных компонент

In [72]:
def quant(coeffs, n_quants):
    min_val = np.min(coeffs)
    max_val = np.max(coeffs)
    step = (max_val - min_val) / n_quants
    quantized = np.round((coeffs - min_val) / step).astype(int)
    return quantized

LH_q = quant(LH, 4)
HL_q = quant(HL, 4)
HH_q = quant(HH, 4)

4. Сохраните получившийся массив значений в текстовый файл в порядке LL, LH, HL, HH

In [74]:
def run_length_encode(data):
    encoded = []
    for value, count in Counter(data.flatten()).items():
        encoded.append((value, count))
    return encoded

LH_rle = run_length_encode(LH_q)
HL_rle = run_length_encode(HL_q)
HH_rle = run_length_encode(HH_q)

with open('haar_image.txt', 'w') as f:
    np.savetxt(f, LL, fmt='%d')
    f.write("LH:\n")
    for value, count in LH_rle:
        f.write(f"{value} {count}\n")
    f.write("HL:\n")
    for value, count in HL_rle:
        f.write(f"{value} {count}\n")
    f.write("HH:\n")
    for value, count in HH_rle:
        f.write(f"{value} {count}\n")

haar_storage_size = os.path.getsize('haar_image.txt')

In [75]:
print(f"Исходный размер: {pixel_storage_size} байт")
print(f"Размер после сжатия:: {haar_storage_size} байт")
print(f"Коэффициент сжатия: {pixel_storage_size / haar_storage_size:.2f}")

Исходный размер: 29900774 байт
Размер после сжатия:: 7470812 байт
Коэффициент сжатия: 4.00
